<a href="https://colab.research.google.com/github/colinder/Naver_MovieReview_LSTM/blob/main/Naver_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Naver 영화 리뷰 감정분석**
데이터 셋은 https://github.com/e9t/nsmc 의 자료를 사용

In [70]:
import urllib.request
import pandas as pd

## data 가져오기
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="total_data.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

total_data = pd.read_table("total_data.txt")
train_data = pd.read_table("ratings_train.txt")
test_data = pd.read_table("ratings_test.txt")

print("총 데이터 개수: ", len(total_data))
print("훈련 데이터 개수:", len(train_data))
print("검증 데이터 개수:", len(x_text))

총 데이터 개수:  200000
훈련 데이터 개수: 150000
검증 데이터 개수: 50000


## 불러온 데이터 __확인__

In [71]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


## 작업의 편의를 위해 index 열 추가


In [72]:
train_data.insert(0, "index", [i for i in range(150000)])

In [73]:
train_data.head()

,index,id,document,label
0,0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,2,10265843,너무재밓었다그래서보는것을추천한다,0
3,3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# **데이터 전처리 진행**
데이터의 이상값, 중복 등을 처리하여 정제된 데이터 셋으로 구성
</br>
</br>
# 1. 정규표현식을 사용해 '!'나'.' 등 기호를 삭제
 

In [74]:
train_data['document'] = train_data['document'].str.replace("[^0-9^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

print("정규표현식 적용 후 데이터의 수:", len(train_data))

정규표현식 적용 후 데이터의 수: 150000


<ipython-input-74-205dbd72b237>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^0-9^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [76]:
## 정규표현식 적용 확인
train_data.head()

,index,id,document,label
0,0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,2,10265843,너무재밓었다그래서보는것을추천한다,0
3,3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


# 2. 중복 데이터 확인

In [78]:
# 1. 중복 id 확인
train_data["id"].is_unique        # is_unique ; res => boolean

# 2. 중복 document 확인
train_data["document"].is_unique        # False

# 3. id 별 document 중복 확인
dupl = train_data[   (train_data['id'].is_unique & train_data['document'].duplicated())]
dupl

,index,id,document,label
412,412,9509970,,1
1058,1058,5890638,4,0
1097,1097,2062443,쵝오,1
1165,1165,7508856,최고,1
1248,1248,3502685,최고,1
...,...,...,...,...
149949,149949,3924716,영화,0
149960,149960,6368431,지루하다,0
149966,149966,6735753,1,0
149987,149987,7669621,재미있어요^^,1


## 복데이터 삭제 X
why? <br>내용이 중복되어 있더라도  유일한 사용자가 등록한 정보임으로 개개인의 호,불호를 판단할 수 있음.<br> 
하여 중복데이터의 확인만 진행 후 삭제는 하지 않음.

In [80]:
# 중복 데이터의 비율
print("중복 데이터 비율(Null 제외):", len(dupl)/len(train_data))


중복 데이터 비율(Null 제외): 0.04038


# 3. Null(결측값) 관리

Null값의 형태 및 분포를 파악하고 관리한다.

In [81]:
# 결측값 확인
train_data.isnull().sum()

index       0
id          0
document    5
label       0
dtype: int64

In [82]:
# 결측값 행 확인
train_data[train_data['document'].isnull()]

,index,id,document,label
25857,25857,2172111,NaN,1
55737,55737,6369843,NaN,1
110014,110014,1034280,NaN,0
126782,126782,5942978,NaN,0
140721,140721,1034283,NaN,0


*영화 리뷰 데이터 셋에서 의견은 없으나 긍정, 부정에 대한 마크를 했다는 것은 '자연어 처리'에서는 필요없는 데이터로 판단하여 제거*

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


쵝오 발견 / 인덱스는 1055


,id,document,label
1055,4083134,쵝오,1


In [ ]:
train_data[1053:1056]

,id,document,label
1054,6061784,중2때 극장 모퉁이에서 봤던 영화 그때의 감동을 추억으로 더듬을 수 밖에 없는 쓸쓸함,1
1055,4083134,쵝오,1
1056,6184795,제발 다른 인간들 보고 욕좀해줘라 ㅋㅋ 시간 배우 정말 아깝다,0


In [ ]:
## Null값 판단 및 제거
IsNull_indexs = []
for i, v in enumerate(train_data["document"].isnull()):
  if v == True:
    IsNull_indexs.append(i+1)

print("null값 갯수:", len(IsNull_indexs), "/", 'null값 index', IsNull_indexs)

null값 갯수: 1 / null값 index [25232]


In [ ]:
## null값 확인
train_data[25230:25233]

,id,document,label
25856,9475362,샘 킴 반지좀빼고요리하지 양손어다반지 비위생,0
25857,2172111,NaN,1
25858,8475146,진짜재밋어요 또개봉했으면 좋겠다 또봐야지,1


In [ ]:
# 어떤 행이던지 null 값이 있다면 제거 
train_data = train_data.dropna(how='any')

print("null값 제거 후 데이터 수 :", len(train_data))

null값 제거 후 데이터 수 : 143942


In [ ]:
## null값 삭제 확인
train_data[25230:25233]

,id,document,label
25856,9475362,샘 킴 반지좀빼고요리하지 양손어다반지 비위생,0
25858,8475146,진짜재밋어요 또개봉했으면 좋겠다 또봐야지,1
25859,7783279,벌써 12년 이나 흘러버렸네 추억은 추억일때 아름다운 법 5년 이상 영화는 이제 안...,0


# 3. 데이터 토큰화(벡터화) 진행

(LSTM 분석을 위해서는 벡터화된 데이터셋이 필요함.)</br>
토큰화는 Kkma, Komoran 두가지로 테스트 예정</br>
둘 중 프로젝트에 더 적합한 모습을 보이는 라이브러리로 진행을 위한 테스트


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 6.6MB/s 
     |████████████████████████████████| 460kB 41.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Kkma, Komoran, Okt, Hannanum

Kkma = Kkma()
Komoran = Komoran()
Okt = Okt()
Hannanum = Hannanum()

# print(Okt.normalize("안녕하세욬ㅋㅋㅋ 반가워요 샤릉해"))    # 안녕하세요ㅋㅋㅋ 반가워요 사랑해
# Okt는 정규화 처리 라이브러리가따로 존재한다는 레퍼런스를 보고 정확도에 크게 기여할 수 있을 것같아 테스트 해보았으나, 
# 위와 같은 정규화 처리가 되는 경우가 적은 것으로 판단. (5개 테스트 결과 정규화 처리 건수 0) 
# 이후 너무 느린 속도에 대한 이슈가 없는 Hannanum까지 테스트 진행

for i in range(4):
  print("="*100)
  print(train_data['document'][i])
  print("Kkma", Kkma.morphs(train_data['document'][i]))
  print("Komran",Komoran.morphs(train_data['document'][i]))
  print("Okt", Okt.morphs(train_data['document'][i], norm=True,stem = True))   # norm: 정규화 처리 / stem: 각 단어에서 어간을 추출
  print("Hannanum", Hannanum.morphs(train_data['document'][i]))


안녕하세요ㅋㅋㅋ 반가워요 사랑해
아 더빙 진짜 짜증나네요 목소리
Kkma ['아', '아', '더빙', '진짜', '짜증나', '네요', '목소리']
Komran ['아', '더빙', '진짜', '짜증', '나', '네요', '목소리']
Okt ['아', '더빙', '진짜', '짜증나다', '목소리']
Hannanum ['아', '더빙', '진짜', '짜증나', '네', '요', '목소리']
흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
Kkma ['흠', '포스터', '보고', '초', '딩', '영화', '줄', '오버', '연기', '조차', '가볍', '지', '않', '구나']
Komran ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍', '지', '않', '구나']
Okt ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']
Hannanum ['흠포스터보', '이', '고', '초딩영화줄오버연기조차', '가볍', '지', '않', '구나']
너무재밓었다그래서보는것을추천한다
Kkma ['너무', '재', '밓', '어', '었', '다', '그래서', '보', '는', '것', '을', '추천', '하', 'ㄴ다']
Komran ['너무재밓었다그래서보는것을추천한다']
Okt ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '한', '다']
Hannanum ['너무재밓었다그래서보는것을추천한다']
교도소 이야기구먼 솔직히 재미는 없다평점 조정
Kkma ['교도소', '이야기', '구', '멀', 'ㄴ', '솔직히', '재미', '는', '없', '다', '평점', '조정']
Komran ['교도소', '이야기', '이', '구먼', '솔직히', '재미', '는', '없', '다', '평점', '조정']
Okt ['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조

_테스트 결과 띄어쓰기에 대한 이슈를 뛰어넘어 가장 작은 단위로 분석이 가능했던 Kkma로 진행하는 것이 타당해 보임_

**고려사항**</br>
정교하게 형태소 분석을 수행하지만, 너무 잘게 쪼개져서 오히려 의미분석을 하기 힘들 수 있음.</br>
하지만 비정형데이터를 다루는데 있어 띄어쓰기를 인식하지 못하는 것은 치명적인 문제가 될 수 있을 것으로 판단.

하여 최종적으로 Kkma 와 Okt 두 방법에 대한 효율을 고민해보는 것으로 기획</br>
먼저 Kkma로 개발.

# 3-1불용어 사전을 만들기 위해 '품사tagging'을 진행

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']   # 불용어 사전
# 불용어 사전의 경우 Kkma.pos()로 형태소를 분류하여 직접 제작할 수 있음.
# 해당 진행은 테스트를 위해 간단히만 제작하여 진행.

# train_data["document"][:10]

x_train = []
error_sentence = []
for i, sentence in enumerate(train_data['document']):
  # print("sentence",sentence)
  try:
    temp_x = Kkma.morphs(sentence) # 토큰화
    # print("temp_x", temp_x)
    temp_x = [word for word in temp_x if not word in stopwords] # 불용어 제거
    x_train.append(temp_x)
  except:
    error_sentence.append(i)
    pass

x_train[:10]
print("에러 발생 수:", len(error_sentence)
